In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
import os

In [2]:
df = pd.read_csv("../data/oslobysykkel-2025.csv")
df.head()

,started_at,ended_at,duration,start_station_id,start_station_name,start_station_description,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_description,end_station_latitude,end_station_longitude
0,2025-06-01 03:00:18.285000+00:00,2025-06-01 03:13:57.293000+00:00,819,555,Griffenfeldts gate,ved Colletts gate,59.933703,10.751930,572,Skøyen,under broen,59.922269,10.679580
1,2025-06-01 03:00:20.204000+00:00,2025-06-01 03:16:36.936000+00:00,976,735,Oslo Hospital,ved trikkestoppet,59.903213,10.767344,487,Carl Berners plass nord,ved Hasleveien,59.926929,10.776971
2,2025-06-01 03:03:45.692000+00:00,2025-06-01 03:12:55.683000+00:00,549,485,Sommerfrydhagen,langs Jens Bjelkes gate,59.911453,10.776072,444,AHO,langs Maridalsveien,59.925265,10.750462
3,2025-06-01 03:05:09.876000+00:00,2025-06-01 03:23:25.633000+00:00,1095,387,Studenterlunden,langs Karl Johan,59.914586,10.735453,2330,Stjerneplassen,Krysset Skippergata Storgata,59.913233,10.749959
4,2025-06-01 03:06:09.611000+00:00,2025-06-01 03:12:56.455000+00:00,406,623,7. juni-plassen,langs Henrik Ibsens gate,59.915080,10.730589,2337,Universitetsgata,Hjørnet av Universitetsgata and Kristian IVs Gate,59.915915,10.737835


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,1025963.0,770.474425,850.175764,61.000000,366.000000,564.000000,865.000000,17888.000000
start_station_id,1025963.0,731.631984,750.564761,377.000000,438.000000,501.000000,591.000000,5431.000000
start_station_latitude,1025963.0,59.921159,0.010675,59.898434,59.912713,59.919524,59.928067,59.953411
start_station_longitude,1025963.0,10.746971,0.024395,10.651118,10.730589,10.750847,10.762213,10.814314
end_station_id,1025963.0,746.309637,756.369919,377.000000,442.000000,500.000000,593.000000,5431.000000
end_station_latitude,1025963.0,59.918741,0.009577,59.898434,59.911776,59.916065,59.924732,59.953411
end_station_longitude,1025963.0,10.745458,0.023320,10.651118,10.731219,10.750462,10.760804,10.814314


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025963 entries, 0 to 1025962
Data columns (total 13 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   started_at                 1025963 non-null  object 
 1   ended_at                   1025963 non-null  object 
 2   duration                   1025963 non-null  int64  
 3   start_station_id           1025963 non-null  int64  
 4   start_station_name         1025963 non-null  object 
 5   start_station_description  1025925 non-null  object 
 6   start_station_latitude     1025963 non-null  float64
 7   start_station_longitude    1025963 non-null  float64
 8   end_station_id             1025963 non-null  int64  
 9   end_station_name           1025963 non-null  object 
 10  end_station_description    1025925 non-null  object 
 11  end_station_latitude       1025963 non-null  float64
 12  end_station_longitude      1025963 non-null  float64
dtypes: float64(4

In [5]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

In [6]:
df.isna().sum()

started_at                    0
ended_at                      0
duration                      0
start_station_id              0
start_station_name            0
start_station_description    38
start_station_latitude        0
start_station_longitude       0
end_station_id                0
end_station_name              0
end_station_description      38
end_station_latitude          0
end_station_longitude         0
dtype: int64

In [7]:
df = df.dropna()
df.isna().sum()

started_at                   0
ended_at                     0
duration                     0
start_station_id             0
start_station_name           0
start_station_description    0
start_station_latitude       0
start_station_longitude      0
end_station_id               0
end_station_name             0
end_station_description      0
end_station_latitude         0
end_station_longitude        0
dtype: int64

In [8]:
def formating_time(df):
    df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
    df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')

    df['start_time'] = (df['started_at'].dt.hour)

    df['start_dayofweek'] = df['started_at'].dt.day_name()
    df['start_dayofweek'] = df['start_dayofweek'].map(
        {'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6}
    )
    # return df
formating_time(df)

In [9]:
def time_recovering(df):
  # for checking
  df['started_at_original'] = df['started_at'].copy()
  df['ended_at_original'] = df['ended_at'].copy()

  df.loc[df['ended_at'].isna() & df['started_at'].notna() & df['duration'].notna(), 'ended_at'] = \
    df['started_at'] + pd.to_timedelta(df['duration'], unit='s')

  df.loc[df['started_at'].isna() & df['ended_at'].notna() & df['duration'].notna(), 'started_at'] = \
    df['ended_at'] - pd.to_timedelta(df['duration'], unit='s')

  df['is_time_recovered'] = (
    (df['started_at_original'].isna() & df['started_at'].notna()) |
    (df['ended_at_original'].isna() & df['ended_at'].notna())
    )

  df.drop(columns=['started_at_original', 'ended_at_original'], inplace=True)

  recovered = df[df['is_time_recovered']]
  return recovered.head(3)

In [10]:
time_recovering(df)
formating_time(df)

In [11]:
df.drop(
    columns=['is_time_recovered',
             'started_at',
             'ended_at',
             'start_station_name',
             'start_station_description',
             'end_station_description',
             'end_station_name'
            ],
    inplace=True)

In [12]:
df = df.dropna()
df.isna().sum()

duration                   0
start_station_id           0
start_station_latitude     0
start_station_longitude    0
end_station_id             0
end_station_latitude       0
end_station_longitude      0
start_time                 0
start_dayofweek            0
dtype: int64

In [13]:
# Lọc các chuyến đi bất thường
suspicious_trips = df[
    (df['start_station_id'] == df['end_station_id'])
]
df.drop(suspicious_trips.index, inplace=True)

In [14]:
df = df[
    (df['duration'] > 200) &
    (df['duration'] < 7500)
]

In [15]:
df.to_csv("../data/oslobysykkel-2025-cleaned.csv", index=False)